In [1]:
import cv2


def get_contours_with_filter(image_path):
    """
    讀取影像，進行二值化並篩選出面積最大的輪廓。
    """
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return None, []

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, dst_binary = cv2.threshold(src_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 找到所有輪廓
    contours, _ = cv2.findContours(dst_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"在 {image_path} 中沒有找到任何輪廓。")
        return src, []
    
    # 找到面積最大的輪廓
    max_area_contour = max(contours, key=cv2.contourArea)
    
    # 這裡只回傳面積最大的輪廓，作為比對的唯一參考
    return src, [max_area_contour]


# 載入並處理模板影像
template_path = "template.jpg"
template_img, template_contours = get_contours_with_filter(template_path)

# 載入並處理待比對影像
exercise_path = "exercise2.jpg"
exercise_img, exercise_contours = get_contours_with_filter(exercise_path)


if template_img is None or exercise_img is None:
    # 處理影像載入失敗的情況
    exit()
    
# 如果模板沒有輪廓，直接結束
if not template_contours:
    print("模板影像沒有找到有效的輪廓。")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# 使用模板中面積最大的輪廓作為比對基準
template_contour = template_contours[0]

# 在待比對影像上標示所有找到的輪廓編號
font = cv2.FONT_HERSHEY_SIMPLEX
n = len(exercise_contours)
for i in range(n):
    # 獲取輪廓的質心或第一個點來標註文字
    if len(exercise_contours[i]) > 0:
        cv2.putText(exercise_img, str(i), exercise_contours[i][0][0], font, 1, (0, 0, 255), 2)


# --------------------- 輪廓匹配 ---------------------
# 設定相似度閾值，值越小代表越相似
match_threshold = 0.3
matched_count = 0

print("正在進行輪廓比對...")
for i in range(n):
    match = cv2.matchShapes(template_contour, exercise_contours[i], 1, 0.0)
    print(f"輪廓 {i} 相比對結果為 = {match:.4f}")

    if match < match_threshold:
        # 繪製符合條件的輪廓，使用綠色填充
        cv2.drawContours(exercise_img, exercise_contours, i, (0, 255, 0), -1)
        matched_count += 1
        print(f"✅ 輪廓 {i} 符合圖樣！")

if matched_count == 0:
    print("\n沒有找到符合圖樣的輪廓。")
else:
    print(f"\n找到 {matched_count} 個符合圖樣的輪廓。")


# 顯示結果
cv2.imshow("Template", template_img)
cv2.imshow("Result", exercise_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

正在進行輪廓比對...
輪廓 0 相比對結果為 = 0.2785
✅ 輪廓 0 符合圖樣！

找到 1 個符合圖樣的輪廓。


In [2]:
import cv2
import numpy as np

def get_contours_from_image(image_path):
    """
    讀取影像，進行二值化並找出所有輪廓。
    """
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return None, []

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, dst_binary = cv2.threshold(src_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 找到所有輪廓
    contours, _ = cv2.findContours(dst_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"在 {image_path} 中沒有找到任何輪廓。")
        return src, []
    
    return src, contours

# 載入並處理模板影像
template_path = "template.jpg"
template_img, template_contours = get_contours_from_image(template_path)

# 載入並處理待比對影像
exercise_path = "exercise2.jpg"
exercise_img, exercise_contours = get_contours_from_image(exercise_path)


if template_img is None or exercise_img is None:
    # 處理影像載入失敗的情況
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# 如果模板或待比對影像沒有輪廓，則結束程式
if not template_contours or not exercise_contours:
    print("至少有一張影像沒有找到有效的輪廓。")
    cv2.imshow("Template", template_img)
    cv2.imshow("Exercise", exercise_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# 使用模板中面積最大的輪廓作為比對基準
template_contour = max(template_contours, key=cv2.contourArea)


# 在待比對影像上標示所有找到的輪廓編號
font = cv2.FONT_HERSHEY_SIMPLEX
n = len(exercise_contours)
for i in range(n):
    if len(exercise_contours[i]) > 0:
        x, y, w, h = cv2.boundingRect(exercise_contours[i])
        cv2.putText(exercise_img, str(i), (x, y - 10), font, 0.8, (0, 0, 255), 2)


# --------------------- 輪廓匹配 ---------------------
# 設定相似度閾值，值越小代表越相似
match_threshold = 0.3
matched_count = 0
result_img = exercise_img.copy()

print("正在進行輪廓比對...")
for i in range(n):
    # 確保輪廓有足夠的點進行比對
    if len(exercise_contours[i]) < 5: 
        continue
        
    match = cv2.matchShapes(template_contour, exercise_contours[i], 1, 0.0)
    print(f"輪廓 {i} 相比對結果為 = {match:.4f}")

    if match < match_threshold:
        # 繪製符合條件的輪廓，使用綠色線條
        cv2.drawContours(result_img, exercise_contours, i, (0, 255, 0), 2)
        matched_count += 1
        print(f"✅ 輪廓 {i} 符合圖樣！")


print(f"\n找到 {matched_count} 個符合圖樣的輪廓。")


# 顯示結果
cv2.imshow("Template", template_img)
cv2.imshow("Result", result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

正在進行輪廓比對...
輪廓 4 相比對結果為 = 0.2770
✅ 輪廓 4 符合圖樣！
輪廓 5 相比對結果為 = 0.3183
輪廓 6 相比對結果為 = 0.2770
✅ 輪廓 6 符合圖樣！
輪廓 12 相比對結果為 = 0.2610
✅ 輪廓 12 符合圖樣！
輪廓 15 相比對結果為 = 0.2954
✅ 輪廓 15 符合圖樣！
輪廓 17 相比對結果為 = 0.2770
✅ 輪廓 17 符合圖樣！
輪廓 20 相比對結果為 = 0.3183
輪廓 28 相比對結果為 = 0.2770
✅ 輪廓 28 符合圖樣！
輪廓 31 相比對結果為 = 0.3030
輪廓 32 相比對結果為 = 0.2785
✅ 輪廓 32 符合圖樣！
輪廓 33 相比對結果為 = 0.2586
✅ 輪廓 33 符合圖樣！
輪廓 34 相比對結果為 = 0.1945
✅ 輪廓 34 符合圖樣！

找到 9 個符合圖樣的輪廓。


In [1]:
import cv2
import numpy as np

def get_contours_from_image(image_path):
    """
    讀取影像，進行二值化並找出所有輪廓。
    """
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return None, []

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, dst_binary = cv2.threshold(src_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 找到所有輪廓
    contours, _ = cv2.findContours(dst_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"在 {image_path} 中沒有找到任何輪廓。")
        return src, []
    
    return src, contours

# 載入並處理模板影像
template_path = "template.jpg"
template_img, template_contours = get_contours_from_image(template_path)

# 載入並處理待比對影像
exercise_path = "exercise2.jpg"
exercise_img, exercise_contours = get_contours_from_image(exercise_path)


if template_img is None or exercise_img is None:
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

if not template_contours or not exercise_contours:
    print("至少有一張影像沒有找到有效的輪廓。")
    cv2.imshow("Template", template_img)
    cv2.imshow("Exercise", exercise_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()


# 使用模板中面積最大的輪廓作為比對基準
template_contour = max(template_contours, key=cv2.contourArea)
template_area = cv2.contourArea(template_contour)


# 在待比對影像上標示所有找到的輪廓編號
font = cv2.FONT_HERSHEY_SIMPLEX
n = len(exercise_contours)
for i in range(n):
    if len(exercise_contours[i]) > 0:
        x, y, w, h = cv2.boundingRect(exercise_contours[i])
        cv2.putText(exercise_img, str(i), (x, y - 10), font, 0.8, (0, 0, 255), 2)


# --------------------- 整合面積與形狀比對 ---------------------
# 設定相似度閾值，值越小代表越相似
match_threshold = 0.3
# 設定面積差異閾值（百分比），例如 0.5 代表允許面積差異在 50% 以內
area_threshold_factor = 0.5 

matched_count = 0
result_img = exercise_img.copy()

print("正在進行輪廓比對...")
for i in range(n):
    # 確保輪廓有足夠的點進行比對
    if len(exercise_contours[i]) < 5: 
        continue
    
    current_contour = exercise_contours[i]
    current_area = cv2.contourArea(current_contour)
    
    # 1. 進行面積篩選：檢查當前輪廓的面積是否在可接受的範圍內
    # 這裡使用面積差異的絕對值來判斷
    area_diff = abs(current_area - template_area) / template_area
    
    # 2. 進行形狀比對：如果面積符合，才進行 shape 比對
    if area_diff < area_threshold_factor:
        match = cv2.matchShapes(template_contour, current_contour, 1, 0.0)
        print(f"輪廓 {i} 相比對結果為 = {match:.4f} (面積差異: {area_diff:.2f})")
        
        if match < match_threshold:
            # 繪製符合條件的輪廓，使用綠色線條
            cv2.drawContours(result_img, exercise_contours, i, (0, 255, 0), 2)
            matched_count += 1
            print(f"✅ 輪廓 {i} 符合圖樣！")
    else:
        print(f"輪廓 {i} 面積差異過大 (面積差異: {area_diff:.2f})，跳過比對。")


print(f"\n找到 {matched_count} 個符合圖樣的輪廓。")


# 顯示結果
cv2.imshow("Template", template_img)
cv2.imshow("Result", result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

正在進行輪廓比對...
輪廓 4 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 5 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 6 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 12 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 15 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 17 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 20 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 28 面積差異過大 (面積差異: 1.00)，跳過比對。
輪廓 31 面積差異過大 (面積差異: 1.46)，跳過比對。
輪廓 32 面積差異過大 (面積差異: 1.78)，跳過比對。
輪廓 33 面積差異過大 (面積差異: 1.17)，跳過比對。
輪廓 34 面積差異過大 (面積差異: 1.46)，跳過比對。

找到 0 個符合圖樣的輪廓。


In [2]:
import cv2
import numpy as np


def get_contours_from_image(image_path):
    """
    讀取影像，進行二值化並找出所有輪廓。
    """
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return None, []

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, dst_binary = cv2.threshold(src_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 這裡的 cv2.RETR_TREE 會回傳所有輪廓及其層次關係
    contours, hierarchy = cv2.findContours(dst_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"在 {image_path} 中沒有找到任何輪廓。")
        return src, [], None
    
    return src, contours, hierarchy[0]  # hierarchy 是三維陣列，我們只需要第一個維度


def get_top_level_contours(contours, hierarchy, min_area_threshold=1000):
    """
    篩選出頂層（沒有父輪廓）且面積大於閾值的輪廓。
    """
    top_level_contours = []
    
    for i, contour in enumerate(contours):
        # 檢查輪廓是否有父輪廓 (hierarchy[i][3] == -1 表示沒有父輪廓)
        if hierarchy[i][3] == -1:
            area = cv2.contourArea(contour)
            # 檢查面積是否大於閾值
            if area > min_area_threshold:
                top_level_contours.append(contour)
    
    return top_level_contours


# --------------------------------------------------------
# 載入並處理模板影像
template_path = "template.jpg"
template_img, all_template_contours, template_hierarchy = get_contours_from_image(template_path)

# 篩選出模板的主要輪廓
template_main_contours = get_top_level_contours(all_template_contours, template_hierarchy)


# 載入並處理待比對影像
exercise_path = "exercise2.jpg"
exercise_img, all_exercise_contours, exercise_hierarchy = get_contours_from_image(exercise_path)

# 篩選出待比對影像中所有主要輪廓
exercise_main_contours = get_top_level_contours(all_exercise_contours, exercise_hierarchy)


if template_img is None or exercise_img is None:
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()


# 確保模板有主要輪廓可以比對
if not template_main_contours:
    print("模板影像沒有找到有效的頂層輪廓。")
    cv2.imshow("Template", template_img)
    cv2.imshow("Exercise", exercise_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# 使用模板中面積最大的頂層輪廓作為比對基準
template_contour = max(template_main_contours, key=cv2.contourArea)


# --------------------- 輪廓比對 ---------------------
# 設定形狀相似度閾值，值越小代表越相似
match_threshold = 0.3
matched_count = 0
result_img = exercise_img.copy()

print("正在進行輪廓比對...")
for i, contour in enumerate(exercise_main_contours):
    # 確保輪廓有足夠的點進行比對
    if len(contour) < 5: 
        continue
    
    match = cv2.matchShapes(template_contour, contour, 1, 0.0)
    print(f"頂層輪廓 {i} 相比對結果為 = {match:.4f}")

    if match < match_threshold:
        # 繪製符合條件的輪廓，使用綠色線條
        cv2.drawContours(result_img, exercise_main_contours, i, (0, 255, 0), 2)
        matched_count += 1
        print(f"✅ 輪廓 {i} 符合圖樣！")


print(f"\n找到 {matched_count} 個符合圖樣的輪廓。")


# 顯示結果
cv2.imshow("Template", template_img)
cv2.imshow("Result", result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

正在進行輪廓比對...
頂層輪廓 0 相比對結果為 = 0.3030
頂層輪廓 1 相比對結果為 = 0.2785
✅ 輪廓 1 符合圖樣！
頂層輪廓 2 相比對結果為 = 0.2586
✅ 輪廓 2 符合圖樣！
頂層輪廓 3 相比對結果為 = 0.1945
✅ 輪廓 3 符合圖樣！

找到 3 個符合圖樣的輪廓。


In [3]:
import cv2
import numpy as np


def get_contours_from_image(image_path):
    """
    讀取影像，進行二值化並找出所有輪廓。
    """
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return None, []

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, dst_binary = cv2.threshold(src_gray, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    contours, hierarchy = cv2.findContours(dst_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print(f"在 {image_path} 中沒有找到任何輪廓。")
        return src, [], None
    
    return src, contours, hierarchy[0]


def get_filtered_contours(contours, hierarchy, min_area_threshold=1000):
    """
    篩選出頂層（沒有父輪廓）且面積大於閾值的輪廓。
    """
    filtered_contours = []
    
    for i, contour in enumerate(contours):
        if hierarchy[i][3] == -1:  # 檢查是否為頂層輪廓
            area = cv2.contourArea(contour)
            if area > min_area_threshold:
                filtered_contours.append(contour)
    
    return filtered_contours


# 載入並處理模板影像
template_path = "template.jpg"
template_img, template_contours, template_hierarchy = get_contours_from_image(template_path)
template_main_contours = get_filtered_contours(template_contours, template_hierarchy)


# 載入並處理待比對影像
exercise_path = "exercise2.jpg"
exercise_img, exercise_contours, exercise_hierarchy = get_contours_from_image(exercise_path)
exercise_main_contours = get_filtered_contours(exercise_contours, exercise_hierarchy)


if template_img is None or exercise_img is None:
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

if not template_main_contours or not exercise_main_contours:
    print("至少有一張影像沒有找到有效的輪廓。")
    cv2.imshow("Template", template_img)
    cv2.imshow("Exercise", exercise_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()


# 使用模板中面積最大的頂層輪廓作為比對基準
template_contour = max(template_main_contours, key=cv2.contourArea)


# --------------------- 輪廓比對：找出唯一最相似的輪廓 ---------------------
best_match_score = float('inf')  # 初始化為無限大
best_match_contour = None
best_match_index = -1
result_img = exercise_img.copy()

print("正在進行輪廓比對...")
for i, contour in enumerate(exercise_main_contours):
    if len(contour) < 5:
        continue
    
    match_score = cv2.matchShapes(template_contour, contour, 1, 0.0)
    print(f"輪廓 {i} 相比對結果為 = {match_score:.4f}")

    # 如果找到比目前最佳分數更低（更相似）的輪廓，就更新
    if match_score < best_match_score:
        best_match_score = match_score
        best_match_contour = contour
        best_match_index = i

# --------------------- 顯示結果 ---------------------
if best_match_contour is not None:
    # 繪製最佳匹配的輪廓，並使用綠色填充
    cv2.drawContours(result_img, [best_match_contour], -1, (0, 255, 0), -1)
    print(f"\n✅ 找到最相似的輪廓：輪廓 {best_match_index}，相似度分數為 {best_match_score:.4f}")
else:
    print("\n沒有找到任何符合條件的輪廓。")


cv2.imshow("Template", template_img)
cv2.imshow("Result", result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

正在進行輪廓比對...
輪廓 0 相比對結果為 = 0.3030
輪廓 1 相比對結果為 = 0.2785
輪廓 2 相比對結果為 = 0.2586
輪廓 3 相比對結果為 = 0.1945

✅ 找到最相似的輪廓：輪廓 3，相似度分數為 0.1945
